In [30]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
%run classifiers.ipynb
%run helpers.ipynb

In [31]:
amt_data = pd.read_csv('big_amt_updated.csv')
amt_data.head()

C:\Users\Samarth\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,task_id,group_id,worker,requester,status,requester_response,start_time,end_time,work_time,pay,...,worker_qualifications,group_hourly_rate,dropped_hits,finished_hits,turkopticon_comm,turkopticon_fair,turkopticon_fast,turkopticon_pay,requester_rejected_hits,requester_approved_hits
0,3XIQGXAUMC70NGIUL73D9JZC94R7XF,3ACJON8MIPK8IYPTP8UEHK6MY26EAX,31,A3OSXTUM1QEXNY,submitted,NaN,2016-04-09 16:27:47.752549-04,2016-04-09 16:36:14.911241-04,507,1.00,...,0,1545.06,250,335,3.64,4.47,4.23,3.36,4004,4
1,3LEIZ60CDJYT6U621MPIK06TDHT9ZH,3ACJON8MIPK8IYPTP8UEHK6MZVDEAR,31,A3OSXTUM1QEXNY,submitted,NaN,2016-04-29 18:47:42.062042-04,2016-04-29 20:21:18.787766-04,5616,1.00,...,0,124.61,250,335,3.64,4.47,4.23,3.36,4004,4
2,33M4IA01QHF0GAHN6D7GMURKCGHRX6,33PEMSEYX7H9P6A3OL7VJWN3NXR2EF,34,A3F75T2OL5SPPL,approved,NaN,2015-03-25 21:55:08.213167-04,2015-03-25 21:55:10.160023-04,2,0.02,...,468,36.92,226,824,5.00,5.00,5.00,4.95,1,2
3,3X73LLYYQ1EOBIVKBXLADJQ8XF2HNT,34RF8KPUOTTAJYLLB5X8C0MISJ0BSF,34,A12ZL58R99R5X9,submitted,NaN,2015-05-21 10:13:15.880296-04,2015-05-21 10:16:06.971282-04,171,0.60,...,468,12.62,226,824,3.50,5.00,4.93,3.54,471,0
4,37TRT2X24R83SPP0NFTWJYPI7BWBJI,369VX4MEPKS27BENDM5JU7QLEMEIGC,34,AIHZ2O84RRHN2,submitted,NaN,2015-04-23 15:54:26.672669-04,2015-04-23 16:01:41.67431-04,435,3.50,...,468,28.97,226,824,5.00,5.00,5.00,4.40,4,0


In [32]:
"""Data preprocessing"""
# Remove rows with -ve work_time
amt_data = remove_negatives_from_column(data=amt_data,column_name='work_time')
# Remove rows with turkopticon values Nan
amt_data = remove_null_from_column(data=amt_data, column_name='turkopticon_comm')
# Add pay_per_hour column
amt_data = add_pay_per_hr_column(data=amt_data, column_name='pay_per_hr')

In [33]:
# Remove columns that are not required
num_columns = len(amt_data.columns)
col_to_keep = [8,9] + list(range(11,num_columns))
amt_reduced = amt_data.iloc[:,col_to_keep]
amt_reduced.columns

Index(['work_time', 'pay', 'worker_qualifications', 'group_hourly_rate',
       'dropped_hits', 'finished_hits', 'turkopticon_comm', 'turkopticon_fair',
       'turkopticon_fast', 'turkopticon_pay', 'requester_rejected_hits',
       'requester_approved_hits', 'pay_per_hr'],
      dtype='object')

In [34]:
amt_reduced.head()

,work_time,pay,worker_qualifications,group_hourly_rate,dropped_hits,finished_hits,turkopticon_comm,turkopticon_fair,turkopticon_fast,turkopticon_pay,requester_rejected_hits,requester_approved_hits,pay_per_hr
0,507,1.00,0,1545.06,250,335,3.64,4.47,4.23,3.36,4004,4,7.100592
1,5616,1.00,0,124.61,250,335,3.64,4.47,4.23,3.36,4004,4,0.641026
2,2,0.02,468,36.92,226,824,5.00,5.00,5.00,4.95,1,2,36.000000
3,171,0.60,468,12.62,226,824,3.50,5.00,4.93,3.54,471,0,12.631579
4,435,3.50,468,28.97,226,824,5.00,5.00,5.00,4.40,4,0,28.965517


In [35]:
amt_reduced = filter_data_by_column(data=amt_reduced, column_name='work_time', value=3000, logic='less_than')
#amt_reduced.plot(kind="scatter", x="work_time", y="pay_per_hr", alpha=0.1)

In [36]:
# Removing all entries with pay_per_hr>100 (Outliers) (Although 100$ per hr is itself very high)
amt_pay_filter1 = filter_data_by_column(data=amt_reduced, column_name='pay_per_hr', value=50, logic='less_than')
print("Entries: {0}".format(len(amt_pay_filter1)))
# Creating scatter plot
#amt_pay_filter1.plot(kind="scatter", x="work_time", y="pay_per_hr", alpha=0.1)

Entries: 161320


In [37]:
# Since the work_time values range from 1 to ~65000, it is hard to classify with their original values.
# Hence, we will be taking log of the work_time values
amt_data_log = amt_reduced.copy()
amt_data_log['work_time'] = np.log10(amt_reduced['work_time'])

In [38]:
# Create inputs and labels from the dataset
num_of_columns = len(amt_data_log.columns)
X = amt_data_log.iloc[:,2:num_of_columns]
y_temp = amt_data_log.iloc[:,0]
y_temp.head()

0    2.705008
2    0.301030
3    2.232996
4    2.638489
5    1.799341
Name: work_time, dtype: float64

In [39]:
X['pay_per_hr']=create_bins(3,X['pay_per_hr'])

C:\Users\Samarth\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\Samarth\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\Samarth\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [40]:
# Categorise work_time
"""
0-1 ----> Class 0
1-2 ----> Class 1
2-3 ----> Class 2
3-4 ----> Class 3
4-5 ----> Class 4
"""
y = np.floor(y_temp)
y = y.astype('int')
y.value_counts()

1    82844
0    50395
2    29006
3     2083
Name: work_time, dtype: int64

In [41]:
X['pay_per_hr'].value_counts()
X.head()

,worker_qualifications,group_hourly_rate,dropped_hits,finished_hits,turkopticon_comm,turkopticon_fair,turkopticon_fast,turkopticon_pay,requester_rejected_hits,requester_approved_hits,pay_per_hr
0,0,1545.06,250,335,3.64,4.47,4.23,3.36,4004,4,1.0
2,468,36.92,226,824,5.00,5.00,5.00,4.95,1,2,2.0
3,468,12.62,226,824,3.50,5.00,4.93,3.54,471,0,2.0
4,468,28.97,226,824,5.00,5.00,5.00,4.40,4,0,2.0
5,468,17.37,226,824,5.00,4.93,4.25,2.81,300,1,2.0


In [42]:
# One hot encoding of pay_per_hr
from sklearn.preprocessing import OneHotEncoder
one_hot_encoder = OneHotEncoder(categorical_features = [10])
X_array = one_hot_encoder.fit_transform(X).toarray()
X_array = X_array[:,1:]
X_encoded = pd.DataFrame(data = X_array, columns=['pay_per_hr_1','pay_per_hr_2']+list(X.columns[1:]))

In [43]:
X_encoded = X_encoded.iloc[:,0:-1]

In [44]:
X_encoded.head()

,pay_per_hr_1,pay_per_hr_2,group_hourly_rate,dropped_hits,finished_hits,turkopticon_comm,turkopticon_fair,turkopticon_fast,turkopticon_pay,requester_rejected_hits,requester_approved_hits
0,1.0,0.0,0.0,1545.06,250.0,335.0,3.64,4.47,4.23,3.36,4004.0
1,0.0,1.0,468.0,36.92,226.0,824.0,5.00,5.00,5.00,4.95,1.0
2,0.0,1.0,468.0,12.62,226.0,824.0,3.50,5.00,4.93,3.54,471.0
3,0.0,1.0,468.0,28.97,226.0,824.0,5.00,5.00,5.00,4.40,4.0
4,0.0,1.0,468.0,17.37,226.0,824.0,5.00,4.93,4.25,2.81,300.0


In [45]:
# Split data into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

In [46]:
# Feature normalization
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [47]:
# Run Decision tree
run_decision_tree(X_train, y_train, X_test, y_test)

DT Test Score: 0.880484391164121
DT Train Score: 0.8992332385023809
Confusion Matrix: [[ 9578   475     5     2]
 [  316 15168  1067    13]
 [   27  1606  4086    99]
 [    3    43   272   106]]


In [49]:
def run_random_forest(X_train, y_train, X_test, y_test):
    rfc = RandomForestClassifier(n_estimators=100)
    rfc.fit(X_train, y_train)
    train_score = rfc.score(X_train, y_train)
    test_score = rfc.score(X_test, y_test)
    print("RF Test Score: {0}".format(test_score))
    print("RF Train Score: {0}".format(train_score))
    rfc_y_pred = rfc.predict(X_test)
    confusion_matrix = metrics.confusion_matrix(y_test, rfc_y_pred)
    print("RF Confusion Matrix: {0}".format(confusion_matrix))
    feature_importances = pd.DataFrame(rfc.feature_importances_, index = X_encoded.columns[:],
                                    columns=['importance']).sort_values('importance',ascending=False)
    print("RF Feature Importance: {0}".format(feature_importances))
    return (train_score, test_score, rfc_y_pred, confusion_matrix, feature_importances, rfc)

In [50]:
# Random Forest classifier
run_random_forest(X_train, y_train, X_test, y_test)

RF Test Score: 0.8835270492302075
RF Train Score: 0.8992256317414918
RF Confusion Matrix: [[ 9574   482     4     0]
 [  309 15162  1088     5]
 [    7  1545  4203    63]
 [    1    33   291    99]]
RF Feature Importance:                          importance
pay_per_hr_2               0.170377
dropped_hits               0.166826
pay_per_hr_1               0.137955
requester_approved_hits    0.093704
turkopticon_pay            0.084156
requester_rejected_hits    0.078189
finished_hits              0.069469
turkopticon_fair           0.068952
turkopticon_comm           0.061007
turkopticon_fast           0.057102
group_hourly_rate          0.012262


(0.8992256317414918,
 0.8835270492302075,
 array([0, 1, 1, ..., 0, 1, 0]),
 array([[ 9574,   482,     4,     0],
        [  309, 15162,  1088,     5],
        [    7,  1545,  4203,    63],
        [    1,    33,   291,    99]], dtype=int64),
                          importance
 pay_per_hr_2               0.170377
 dropped_hits               0.166826
 pay_per_hr_1               0.137955
 requester_approved_hits    0.093704
 turkopticon_pay            0.084156
 requester_rejected_hits    0.078189
 finished_hits              0.069469
 turkopticon_fair           0.068952
 turkopticon_comm           0.061007
 turkopticon_fast           0.057102
 group_hourly_rate          0.012262,
 RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
             max_depth=None, max_features='auto', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_e

In [82]:
X_test.shape

(32866, 11)